In [1]:
# Imports
import pandas as pd
import matplotlib.pyplot as plt
from numpy import genfromtxt
import numpy as np
import pandas as pd
import time
import random
import math

In [2]:
# load (downloaded if needed) the MNIST dataset

train = pd.read_csv('./datasets/train.csv')
test = pd.read_csv('./datasets/test.csv')

#y_train = np.array(df['label'].values)
#print(y_train)

X_train = (train.iloc[:,1:].values.astype('float32'))
y_train = (train.iloc[:,0].values.astype('int32'))
y_train = np.tile(y_train, (1, 1))
X_test = test.values.astype('float32')

In [3]:
print(X_train.shape[1])

784


In [62]:
class Neural_Network:
    def __init__(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train
        self.m = X_train.shape[0]
        self.input_layer_size = X_train.shape[1]
        self.hidden_layer_size = 25
        self.output_layer_size = 10
        self.epsilon = []
        self.theta1 = 0
        self.theta2 = 0
        self.J = 0
        self.grad = 0
        self.num_labels = 10
        self.a1 = 0
        self.a2 = 0
        self.a3 = 0
        
    def randInit(self, X, y):
        ils = self.input_layer_size
        hls = self.hidden_layer_size
        ols = self.output_layer_size
        
        L_in = ils
        L_out = hls
        
        self.epsilon.append(math.sqrt(6) / math.sqrt(L_in + L_out))
        
        L_in = hls
        L_out = ols
        
        self.epsilon.append(math.sqrt(6) / math.sqrt(L_in + L_out))
        
        #The theta values are of the form S(j + 1) x S(j) + 1 where S(j) i sthe size of the layer at j
        
        self.theta1 = np.dot(np.random.rand(hls, ils + 1), (2 * self.epsilon[0])) - self.epsilon[0]
        
        self.theta2 = np.dot(np.random.rand(ols, hls + 1), (2 * self.epsilon[1])) - self.epsilon[1]
        
    def sigmoid(self, z):
        return ( 1/(1 + np.exp(-z)))
    
    def sigmoidPrime(self, z):
        return np.multiply(z, (1-z))
    
    def costFunction(self):
        X = self.X_train
        y = self.y_train
        y = y.T
        
        #Foward propogation
        a1 = np.hstack((np.ones((self.m, 1)), X ))
        self.a1 = a1
        
        z2 = np.dot(a1, np.transpose(self.theta1))
        
        a2 = self.sigmoid(z2)
        a2 = np.hstack((np.ones((a2.shape[0], 1)), a2))
        self.a2 = a2
        
        z3 = np.dot(a2, np.transpose(self.theta2))
        
        a3 = self.sigmoid(z3)
        self.a3 = a3
        
        #Cost function
        
        numLabels_temp = np.tile([i for i in range(10)], (self.m, 1) )
        
        yMatrix_temp = np.tile(y, (1,self.num_labels) )
        
        yMatrix = np.equal(numLabels_temp, yMatrix_temp)
        
        cost_no_reg = np.sum(np.multiply(yMatrix, np.log(a3)) + np.multiply((1-yMatrix), np.log(1 - a3)))
        
        lam = 0
        
        reg = (lam / (2*self.m)) * ((np.sum(np.square(self.theta1[:, 2:None])) + np.sum(np.square(self.theta2[:, 2:None])))) ** 2
        #Also why is it 2:None what does that None do exactly, you
        #seem to be squaring it twice also since you have np.square and 
        #Also I think you should be going from 1: it's 0 we want to avoid because that will
        #have the bias unit
        #then **2 at the end 
        reg_temp_Keanu = (lam / (2*self.m)) * (np.sum(np.sum(self.theta1[:,1:]**2)) +\
                                               np.sum(np.sum(self.theta2[:,1:]**2)))#Keanu
        J =  -(1.0/self.m) * (cost_no_reg + reg)#Keanu
        #For the cost function I think you need to multiply it by a
        #that negative number to get it positive
        
        print(J)
        
        return J
        
    def backProp(self):
        X = self.X_train
        y = self.y_train
        y = y.T
        lam = 0
        
        numLabels_temp = np.tile([i for i in range(10)], (self.m, 1) )
        
        yMatrix_temp = np.tile(y, (1,self.num_labels) )
        
        yMatrix = np.equal(numLabels_temp, yMatrix_temp)
        
        delta2 = 0
        delta1 = 0
        
        for i in range(0, self.m):
            
            a1_t = self.a1[i,]
            #I think here you're adding two set's of one's what,
            #what I've seen you have to add it after you do the dot 
            #product you might want to use a new variable
            #because you're just going to be using parts of a1
            #and you don't need to transpose it
            a2_t = self.a2[i,].T
            a3_t = self.a3[i,].T
        
            z2_t = np.dot(a1_t, self.theta1.T)
            #I'm not sure where you got the sigmoid function for here
            #

            
            yMatrix_i = yMatrix[i,].T
            
            delta3_i = (a3_t -  yMatrix_i)
            z2_t = np.insert(z2_t, 0, 1, axis=0)
            
            g2 = self.sigmoidPrime(z2_t)
            
            delta2_i = np.multiply(np.dot(self.theta2.T, delta3_i), g2)#hmm
            delta2_i = delta2_i[1:]
            
            delta2 = delta2 + np.outer(delta3_i.T, a2_t)
            
            delta1 = delta1 + np.outer(delta2_i.T, a1_t.T)
            
        theta1_grad = delta1 / self.m
        theta2_grad = delta2 / self.m
        
        theta1_grad_unreg = np.copy(theta1_grad)
        theta2_grad_unreg = np.copy(theta2_grad)
        
        #I'm not sure that's how you get no bias
        
        theta1_grad += (float(lam/self.m) * self.theta1)
        theta2_grad += (float(lam/self.m) * self.theta2)
        
        theta1_grad[:,0] = theta1_grad_unreg[:,0]
        theta2_grad[:,0] = theta2_grad_unreg[:,0]
        
        print(theta1_grad)

In [63]:
nn = Neural_Network(X_train, y_train)
nn.randInit(X_train, y_train)
nn.costFunction()
nn.backProp()

7.12391356499474
[[ -286.39242865     0.             0.         ...     0.
      0.             0.        ]
 [-3989.27443795     0.             0.         ...     0.
      0.             0.        ]
 [-6176.08278271     0.             0.         ...     0.
      0.             0.        ]
 ...
 [ 3361.22696902     0.             0.         ...     0.
      0.             0.        ]
 [ 8483.5692162      0.             0.         ...     0.
      0.             0.        ]
 [ 1279.06969662     0.             0.         ...     0.
      0.             0.        ]]
